In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error, f1_score, accuracy_score, roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression

In [2]:
def print_regression_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    print(f'MSE = {mse:.2f}, RMSE = {rmse:.2f}')

In [3]:
data = load_boston()
X, y = data['data'], data['target']
x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

# Задание 1

Реализовать функцию, осуществляющую матричные операции для получения theta. посчитать линейную регрессию

In [4]:
def linreg_linear(X, y):
    return np.dot(np.linalg.inv(np.dot(X.T, X)), np.dot(X.T, y))
theta = linreg_linear(x_train, y_train)

# Задание 2

У какого из признаков наибольшее стандартное отклонение, чему оно равно



In [5]:
def max_feature(data):
    table, feature = data['data'], data['feature_names']
    index_of_max_value = np.where(
        table.std(axis=0) == np.amax(table.std(axis=0)))
    return feature[int(index_of_max_value[0][0])], np.amax(X.std(axis=0))

# Задание 3

Обучите регрессию без дополнительного столбца единиц. Какой получился RMSE?

In [6]:
x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)
theta = linreg_linear(x_train, y_train)
y_pred = x_valid.dot(theta)
y_train_pred = x_train.dot(theta)
print_regression_metrics(y_valid, y_pred)
print_regression_metrics(y_train, y_train_pred)

MSE = 45.19, RMSE = 6.72
MSE = 19.35, RMSE = 4.40


# Задание 4

Очистите данные от строк, где значение признака В меньше 50. Какой получился RMSE?


In [7]:
index_of_feature = np.where(data['feature_names'] == 'B')[0][0]
X1 = X[X[:,index_of_feature]>=50]
y1 = y[X[:,index_of_feature]>=50]

x_train, x_valid, y_train, y_valid = train_test_split(X1, y1, test_size=0.2)
theta = linreg_linear(x_train, y_train)
y_pred = x_valid.dot(theta)
y_train_pred = x_train.dot(theta)
print_regression_metrics(y_valid, y_pred)
print_regression_metrics(y_train, y_train_pred)


MSE = 22.71, RMSE = 4.77
MSE = 24.37, RMSE = 4.94


# Задание 5

Нормализуйте признаки и обучите линейную регрессию матричным
методом. Какой получился RMSE?

In [8]:
data = load_boston()
X, y = data['data'], data['target']

X = (X - X.mean(axis=0)) / X.std(axis=0)
X = np.hstack([np.ones(X.shape[0])[:, np.newaxis], X])

x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)
theta = linreg_linear(x_train, y_train)
y_pred = x_valid.dot(theta)
y_train_pred = x_train.dot(theta)
print_regression_metrics(y_valid, y_pred)
print_regression_metrics(y_train, y_train_pred)

MSE = 24.46, RMSE = 4.95
MSE = 21.74, RMSE = 4.66


# Задание 6

Сделайте для градиентного спуска остановку алгоритма, если максимальное из абсолютных значений компонент градиента становится меньше 0.01. Сравните скорость обучения градиентным спуском и матричными операциями. На какой итерации останавливается градиентный спуск?


In [9]:
class LinRegAlgebra():
    def __init__(self):
        self.theta = None
    
    def fit(self, X, y):
        self.theta = np.linalg.inv(X.transpose().dot(X)).dot(X.transpose()).dot(y)
    
    def predict(self, X):
        return X.dot(self.theta)

In [10]:
class RegOptimizer():
    def __init__(self, alpha, n_iters):
        self.theta = None
        self._alpha = alpha
        self._n_iters = n_iters
    
    def gradient_step(self, theta, theta_grad):
        return theta - self._alpha * theta_grad
    
    def grad_func(self, X, y, theta):
        raise NotImplementedError()

    def optimize(self, X, y, start_theta, n_iters):
        theta = start_theta.copy()

        for i in range(n_iters):
            theta_grad = self.grad_func(X, y, theta)
            if np.amax(theta_grad) <= 0.01:
                break
            theta = self.gradient_step(theta, theta_grad)

        return theta, i
    
    def fit(self, X, y):
        m = X.shape[1]
        start_theta = np.ones(m)
        self.theta, i = self.optimize(X, y, start_theta, self._n_iters)

        return i
        
    def predict(self, X):
        raise NotImplementedError()

In [11]:
class LinReg(RegOptimizer):
    def grad_func(self, X, y, theta):
        n = X.shape[0]
        grad = 1. / n * X.transpose().dot(X.dot(theta) - y)

        return grad
    
    def predict(self, X):
        if self.theta is None:
            raise Exception('You should train the model first')
        
        y_pred = X.dot(self.theta)
        
        return y_pred

In [12]:
def print_regression_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    print(f'MSE = {mse:.2f}, RMSE = {rmse:.2f}')
    
def prepare_boston_data():
    data = load_boston()
    X, y = data['data'], data['target']
    # Нормализовать даннные с помощью стандартной нормализации
    X = (X - X.mean(axis=0)) / X.std(axis=0)
    # Добавить фиктивный столбец единиц (bias линейной модели)
    X = np.hstack([np.ones(X.shape[0])[:, np.newaxis], X])
    
    return X, y

In [13]:
X, y = prepare_boston_data()

In [14]:
linreg_alg = LinRegAlgebra()
linreg_alg.fit(X, y)
y_pred = linreg_alg.predict(X)

# Посчитать значение ошибок MSE и RMSE для тренировочных данных
print_regression_metrics(y, y_pred)

MSE = 21.89, RMSE = 4.68


In [15]:
linreg_crit = LinReg(0.2,1000)
i = linreg_crit.fit(X, y)
print('i =', i)
y_pred = linreg_crit.predict(X)

# Посчитать значение ошибок MSE и RMSE для тренировочных данных
print_regression_metrics(y, y_pred)

i = 212
MSE = 21.90, RMSE = 4.68


In [16]:
%timeit linreg_alg.fit(X, y)

47.8 µs ± 5.93 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [17]:
%timeit linreg_crit.fit(X, y)

2.77 ms ± 74.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
